In [9]:
#Dictionaries
item_costs_dict = {"room": 175, "gauze": 25, "consultation": 400, "catheter": 115}
patient_dict= {}
nurse_dict = {}
doctor_dict = {}
patient_dict["0001"] = Patient("0001", "Stefan", "Nguyen", "12", "Hell Ave", "04/20/2025", ["catheter", "rectal implant"])
nurse_dict["nurse1"] = Nurse('nurse1', 'password', 'Sasha', 'Patschke', 1)
doctor_dict["0001"] = Doctor("0001", "Password1", "Drew", "Gordon", "Family Medicine", "5127814115")


In [3]:
#Patient Module
class Patient:
    def __init__(self, patient_id, first_name, last_name, age, address, admit_date, billing_items):
        self.patient_id = patient_id
        self.first_name = first_name
        self.last_name = last_name
        self.__age = age 
        self.address = address
        self.admit_date = admit_date
        self.billing_items = billing_items
        self.line_item_costs = {}
        for item in billing_items:
            if item in item_costs_dict:
                self.line_item_costs[item] = item_costs_dict[item]
            elif item not in item_costs_dict:
                self.line_item_costs[item] = 0
        self.total_bill = sum(self.line_item_costs.values())
        #Meds and Labs
        self.medications = []
        self.labs = []
        self.lab_total = 0

    def get_age(self):
        return self.__age

    def to_string(self):
        return f"{self.patient_id}, {self.first_name}, {self.last_name}, {self.__age}, {self.address}, {self.admit_date}, {self.billing_items}, {self.line_item_costs}, {self.total_bill}"

def new_patient():
    patient_id = input("Patient ID: ")
    first_name = input("First Name: ")
    last_name = input("Last Name: ")
    age = input("Age: ")
    address = input("Address: ")
    admit_date = input("Admit Date: ")
    billing_items = input("Billing Items as a Comma-Seperated List: ").split(",")
    patient_dict[patient_id] = Patient(patient_id, first_name, last_name, age, address, admit_date, billing_items)

    with open("patients.txt", "a") as file:
        file.write(Patient.to_string(patient_dict[patient_id]) + "\n")
    
    print(f"Patient {first_name} {last_name} has been stored in the file.")

def display_info_for_patient_view():
    id_input = input("Patient_ID: ")
    active_user = patient_dict[id_input]
    info = active_user.to_string().split()
    first_name = info[1]
    last_name = info[2]
    print(f"Name: {first_name[0]}***** {last_name[0]}******")
    print("Itemized Bill:")
    for key, value in active_user.line_item_costs.items():
        print(f"   {key.title()} - ${value}")
    if active_user.labs:
        for item in active_user.labs:
            print("   Lab Tests")
            print(f"      {item}")
            print(f"   Lab Total - ${active_user.lab_total}")
    print(f"Total Bill: ${active_user.total_bill}")


In [1]:
#Doctor Module

lab_catalog = {"CBC": 25, "CMP": 45, "Lipid Panel": 30, "HbA1c": 25}

class Doctor:
#Initializes class instance based on which user log into the system and uses the attributes associated with that user_id key.
    def __init__(self, user_id, password, first_name, last_name, specialty, phone):
        self.__user_id = user_id
        self.__password = password
        self.first_name = first_name
        self.last_name = last_name
        self.specialty = specialty
        self.phone = phone
    def get_password(self):
        return self.__password
        
    def login():
        #self.create_dummy_patient()
        print("\n--- Doctor Login ---")
        user_id = input("User ID: ")
        password = input("Password: ")
        if user_id in doctor_dict:
            if doctor_dict[user_id].get_password() == password:
                print(f"Login successful. Welcome Dr. {doctor_dict[user_id].last_name}")
                doctor_dict[user_id].doctor_menu()
                return
        print("Login failed. Try again.")

#Main menu of doctor functions. Tasks 4 and 5 call functions from other modules.
    def doctor_menu(self):
        while True:
            print("\n--- Doctor Menu ---")
            print("1. Create New Account")
            print("2. View Patient Records")
            print("3. Order Labs")
            print("4. Prescribe Medication")
            print("5. Discharge Patient")
            print("6. Logout")
            task = input("Select an option: ")
            if task == "1":
                account_selection = input("Select account type: ")
                if account_selection.lower().strip() == "doctor":
                    self.create_doctor()
                if account_selection.lower().strip() == "nurse":
                    make_new_nurse() #Added from Nurse Module
                if account_selection.lower().strip() == "patient":
                    new_patient() #Added from Patient Module
            elif task == "2":
                self.view_patient_details()
            elif task == "3":
                self.order_labs()
            elif task == "4":
                self.prescribe_medication()
            elif task == "5":
                discharge_patient()
            elif task == "6":
                print("Logging out...")
                break
            else:
                print("Invalid choice. Try again.")
                
    def create_doctor(self):
        print("\n--- Create New Account ---")
        user_id = input("New User ID: ")
        password = input("Password: ")
        first_name = input("First name: ")
        last_name = input("Last name: ")
        specialty = input("Specialty: ")
        phone = input("Phone number: ")
        doctor_dict[user_id] = Doctor(user_id, password, first_name, last_name, specialty, phone)
        print(f"Doctor account '{user_id}' created successfully.")

#Uses the dictionary data to print record report and reformats the keys using .replace and .title to create informative and aesthetic text.
    def view_patient_details(self):
        while True:
            print("\n--- Patient Details ---")
            patient_details_input = input("Enter Patient ID or Q to exit: ")
            if patient_details_input == 'Q' or patient_details_input == 'q':
                print("Exiting Patient Search")
                break
            try:
                target_patient = patient_dict[patient_details_input]
                print(f"Name - {target_patient.last_name},{target_patient.first_name}\nAge - {target_patient.get_age()}\nAddress - {target_patient.address}\nAdmit Date -  {target_patient.admit_date}")
                print("Itemized Bill:")
                for key, value in target_patient.line_item_costs.items():
                    print(f"   {key.title()} - ${value}")
                if target_patient.labs:
                    for item in target_patient.labs:
                        print("   Lab Tests")
                        print(f"      {item}")
                        print(f"   Lab Total - ${target_patient.lab_total}")
                print(f"Total Bill: ${target_patient.total_bill}")
                break
            except KeyError:
                print("No Patient Found. Try again.")
                continue
        print("--------------------\n")    
    
#Adds labs and lab_total for cost as new dictionary keys for the patient dictionary if not already present.
    def order_labs(self):
        print("\n--- Order Labs ---")
        pt_id = input("Enter patient ID: ")
    
        if pt_id in patient_dict:
            patient = patient_dict[pt_id]
            print("\n--- Lab Menu ---")
            for lab, cost in lab_catalog.items():
                print(f'{lab}: ${cost}')
            lab_select = input("Select Lab: ")
            if lab_select in lab_catalog:
                patient.labs.append(lab_select)
                patient.lab_total += lab_catalog[lab_select]
                patient.total_bill += lab_catalog[lab_select]
                print(f"{lab_select} ordered for patient {pt_id}.")
                print(f"New total lab cost: ${patient.lab_total}")
            else:
                print("Invalid lab selection.")
        else:
            print("Patient ID not found.")

    def prescribe_medication(self):
        print("\n--- Prescribe Medication ---")
        while True:
            pt_id = input("Enter patient ID: ")
            if pt_id in patient_dict:
                patient = patient_dict[pt_id]
                medication__name = input("Medication: ")
                dosage = input("Dosage: ")
                route = input("Route: ")
                frequency = input("Frequency: ")
                med = {"Medication": medication__name, "Dosage": dosage, "Route": route, "Frequency": frequency}
                patient.medications.append(med)
                print(f"{medication__name} has been prescribed for Patient ID {pt_id}.")
                break
            else:
                print('No patient found')
                continue
            

In [2]:
#Nurse Module
class Nurse:
    def __init__(self, user_id, password, first_name, last_name, floor):
        self.__user_id = user_id
        self.__password = password
        self.first_name = first_name
        self.last_name = last_name
        self.floor = floor
    def get_user_id(self):
        return self.__user_id
    def get_password(self):
        return self.__password
    def get_password_hidden(self):
        hidden_password = ""
        for letter in self.get_password():
            hidden_password += "*"
        return hidden_password
    
    def login_nurse():
        print("\n--- Nurse Login ---")
        print("Please enter your User ID and Password, or Q to quit")
        while True: 
            user_id_input = input("User ID: ")
            if user_id_input == 'Q' or user_id_input == 'q':
                print("Exiting Nurse Login")
                print("--------------------\n")
                break
            password_input = input("Password: ")
            if user_id_input in nurse_dict:
                active_user = nurse_dict.get(user_id_input)
                if password_input == active_user.get_password():
                    print("Login Accepted")
                    print("--------------------\n")
                    active_user.interface_for_nurse()
                else:
                    print("User ID or Password Incorrect. Try again")
                    continue
            else:
                print("User ID or Password Incorrect. Try again")
                continue
            break

    def interface_for_nurse(self):
        while True:
            print("\n--- Nurse Interface ---")
            print("Welcome to the Nurse Interface")
            print("1. Make New Patient\n2. View Patient Details\n3. Display Nurse Details\n4. Logout")
            interface_input = input("What would you like to do?\n")
            if interface_input == '1':
                self.make_new_patient()
            elif interface_input == '2':
                self.view_patient_details()
            elif interface_input == '3':
                self.display_nurse_info()
            elif interface_input == '4':
                print("--- Closing interface ---")
                break
            else:
                print("Invalid Input")
                continue
    def make_new_patient(self):
        print("\n--- New Patient ---")
        print("Please enter patient information")
        new_patient()
        print("--------------------")
    def display_nurse_info(self):
        print("--- Account Info ---")
        print(f"Name - {self.last_name},{self.first_name}\nUser ID - {self.get_user_id()}\nPassword - {self.get_password_hidden()}\nFloor - {self.floor}")
        print("--------------------")
    def view_patient_details(self):
        while True:
            print("--- Patient Details ---")
            patient_details_input = input("Enter Patient ID or Q to exit: ")
            if patient_details_input == 'Q' or patient_details_input == 'q':
                print("Exiting Nurse Login")
                break
            try:
                target_patient = patient_dict[patient_details_input]
                print(f"Name - {target_patient.last_name},{target_patient.first_name}\nAge - {target_patient.get_age()}\nAddress - {target_patient.address}\nAdmit Date -  {target_patient.admit_date}\nLine Item Costs - {target_patient.line_item_costs}\nTotal Bill -  {target_patient.total_bill}")
                break
            except KeyError:
                print("No Patient Found. Try again.")
                continue
        print("--------------------\n")

def make_new_nurse():
    print("Please enter Nurse Profile information")
    user_id = input("User ID: ")
    password = input("Password: ")
    first_name = input("First Name: ")
    last_name = input("Last Name: ")
    floor = input("Floor Number: ")
    nurse_dict[user_id] = Nurse(user_id, password, first_name, last_name, floor)   


In [4]:
#Lab Tech Module


# UPDATED THE CODE BASED ON THE VARIABLES ON PATIENT MODULE.... 
# Also will proably change more code on final version like to only be able to view the ones doctor approeved or something like that idk..


class LabTechnician:
    def __init__(self):
        pass
    
        # main menu for lab tech module.
        def lab_menu():
            while True:
                print("\n--- Lab Technician Menu ---")
                print("1. View Assigned Lab Tests")
                print("2. Edit Lab Test Info")
                print("3. Exit") # might change to go back menu on final version of code.
                selected_choice = input("Select an option: ")
    
                if selected_choice == "1":
                    self.view_assigned_labs()
                elif selected_choice == "2":
                    self.edit_lab_test()
                elif selected_choice == "3":
                    print("Exiting Lab Technician Menu...")
                    break
                else:
                    print("Invalid choice. Try again.")
    
    def view_assigned_labs(self):
        # views labs for the specific patient by their ID.
        try:
            pid = int(input("Enter Patient ID: "))
            if pid in patient_dict:
                patient = patient_dict[pid]
                print(f"\nPatient: {patient.first_name} {patient.last_name}")
                for key, value in patient.line_item_costs.items():
                    print(f"   {key.title()} - ${value}")
                if patient.labs:
                    for item in patient.labs:
                        print("   Lab Tests")
                        print(f"      {item}")
                        print(f"   Lab Total - ${patient.lab_total}")
                print(f"Total Bill: ${patient.total_bill}")
            else:
                print("Patient not found.")
        except Exception as e:
            print(f"Error: {e}")

    def edit_lab_test(self):
        # this allows the lab technician to edit lab test name or cost.
        try:
            pid = int(input("Enter Patient ID to edit lab info: ")) # show current labs
            if pid in patient_dict:
                patient = patient_dict[pid] # getting the patient object...
                print(f"\nCurrent Lab Info for {patient.first_name} {patient.last_name}:")
                for key, value in patient.line_item_costs.items():
                    print(f"   {key.title()} - ${value}")
                if patient.labs:
                    for item in patient.labs:
                        print("   Lab Tests")
                        print(f"      {item}")
                        print(f"   Lab Total - ${patient.lab_total}")

                
                new_items = input("Enter new Assigned lab name (comma-separated): ")
                new_cost = int(input("Enter new Assigned lab cost: "))
                
                print(f"\nCurrent labs: {patient.billing_items}: ")

                # recalculates total bill form the orginal.... (this part will proably need debugging if it crashes as I has to rush this part...)
                patient.total_bill = new_cost + (patient.total_bill - patient.line_item_costs)
                patient.billing_items = new_items
                patient.line_item_costs = new_cost

                # THIS CODE ONLY IF WE ARE STOING IN txt file......Saves updated patient dict to patients.txt....
                with open("patients.txt", "w") as file:
                    for p in patient_dict.values():
                        file.write(p.to_string() + "\n")

                print("Lab info updated successfully.")
            else:
                print("Patient not found.")
        except Exception as e:
            print(f"Error: {e}")


In [5]:
#Exit Module
import sys
def exit_system():
    print("Exiting HIS. Bye bye! Keep up the amazing work! :)")
    sys.exit()

In [7]:
#Discharge Module

def discharge_patient():
    patient_id = input("Patient ID for Discharge: ")
    if patient_id not in patient_dict:
        print(f"No patient found with ID: {patient_id}")
        return

    patient = patient_dict[patient_id]
    filename = f"{patient.first_name.lower()}{patient.last_name.lower()}_discharged.txt"

    # Displaying details
    print("\n--- Discharge Summary ---")
    print(f"Patient ID: {patient_id}")
    print(f"Name: {patient.first_name} {patient.last_name}")
    for key, value in patient.line_item_costs.items():
        print(f"   {key.title()} - ${value}")
    if patient.labs:
        for item in patient.labs:
            print("   Lab Tests")
            print(f"      {item}")
            print(f"   Lab Total - ${patient.lab_total}")
    print(f"Total Bill: ${patient.total_bill}")
    print("--------------------------")

    # Writing to file
    with open(filename, 'w') as file:
        file.write(f"Patient ID: {patient_id}\n")
        file.write(f"Name: {patient.first_name} {patient.last_name}\n")
        file.write("Itemized Bill:\n")
        for key, value in patient.line_item_costs.items():
            file.write(f"   {key.title()} - ${value}\n")
        if patient.labs:
            print("Lab Tests:\n")
            for item in patient.labs:
                file.write(f"      {item}\n")
        file.write(f"   Lab Total - ${patient.lab_total}\n")
        file.write(f"Total Bill: ${patient.total_bill}\n")

    # Deleting patient record
    print(f"Patient {patient.first_name} {patient.last_name} has been discharged and the record has been deleted.\nFile saved as {filename}.")
    del patient_dict[patient_id]
    

In [10]:
#Main Menu
def open_main_menu():
    print("Welcome to the Hospital Interface System")
    print("Sign In As: \n")
    print("1. Patient\n2. Doctor\n3. Nurse\n4. Lab Technician\n5. Exit System")
    while True:
        main_input = input("")
        if main_input == "1":
            display_info_for_patient_view()
        elif main_input == "2":
            Doctor.login()
        elif main_input == "3":
            Nurse.login_nurse()
        elif main_input == "4":
            LabTechnician.lab_menu()
        elif main_input == "5":
            exit_system()
        else:
            print("Not an option. Try again.")

open_main_menu()

Welcome to the Hospital Interface System
Sign In As: 

1. Patient
2. Doctor
3. Nurse
4. Lab Technician
5. Exit System


 0001


Not an option. Try again.


 1
Patient_ID:  0001


Name: S***** N******
Itemized Bill:
   Catheter - $115
   Rectal Implant - $0
Total Bill: $115


 5


Exiting HIS. Bye bye! Keep up the amazing work! :)


SystemExit: 

C:\Users\Sasha\BMI_110\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
